In [2]:
import pandas as pd
import time
import numpy as np
import json
from pandas.io.json import json_normalize

Import the original csv data and prepare fot the analysis

In [3]:
movies = pd.read_csv("movies.csv")

ratings = pd.read_csv("Ratings.csv")

users = pd.read_csv("Users.csv")


Data Preparation:
    Rating - Convert the timestamp into year
    Movies -  split the released year from the title and separate the genres for each movie

In [1]:
ratings['timestamps'] = ratings['timestamps'].apply(lambda x: time.strftime('%Y', time.localtime(x)))

NameError: name 'ratings' is not defined

In [13]:
movies['year'] = movies.Title.str.extract("\((\d{4})\)", expand=True)
movies.head()


movies_tidy1 = pd.DataFrame(movies.Genres.str.split('|').tolist(), index = movies.MovieID)

movies_tidy2 = movies_tidy1.stack().reset_index()
movies_tidy2.head()

movies_genres = movies_tidy2.drop(columns = ['level_1']).rename(columns = {0 : 'genres' })
movies_genres.head()

movies = pd.merge(movies.drop('Genres', axis = 1), movies_genres)
#movies.to_csv("movies_scatterplot.csv")
#movies_tidy1.to_csv("movie_tidy.csv")

Combined all three sheets into the same table

In [14]:
data = pd.merge(ratings, users)
data = pd.merge(data, movies, left_on = 'MovieID', right_on = 'MovieID', sort = True)

data = data.drop(columns = ['Unnamed: 0_x', 'Unnamed: 5', 'Unnamed: 0_y'])
data.head()

,UserID,MovieID,Rating,timestamps,index,Gender,Age,OccupationID,State,Title,year,genres
0,1,1,5,2001,0,F,1,10,MI,Toy Story (1995),1995,Animation
1,1,1,5,2001,0,F,1,10,MI,Toy Story (1995),1995,Children's
2,1,1,5,2001,0,F,1,10,MI,Toy Story (1995),1995,Comedy
3,6,1,4,2000,5,F,50,9,MN,Toy Story (1995),1995,Animation
4,6,1,4,2000,5,F,50,9,MN,Toy Story (1995),1995,Children's


Calculate the number of user, female/male proportion, average age of each state

In [10]:
state_userinfo = users.pivot_table('UserID', index = ['State'], columns = ['Gender'], aggfunc = 'count')


state_userinfo['TotalUser'] = users.pivot_table('UserID', index = ['State'], aggfunc = 'count')
state_userinfo['Age'] = users.pivot_table('Age', index = ['State'], aggfunc = 'mean')
state_userinfo['Rating'] = data.pivot_table('Rating', index = ['State'], aggfunc = 'mean')

state_userinfo.head()
state_userinfo.to_csv("State_user.csv")


In [344]:
state_userinfo.head()

<bound method DataFrame.info of Gender      F      M  TotalUser        Age    Rating
State                                               
AE        2.0    1.0          3  38.333333  3.661601
AK        5.0    5.0         10  34.700000  4.035714
AL        5.0   18.0         23  28.260870  3.682956
AP        NaN    3.0          3  26.000000  3.921154
AR        7.0   22.0         29  31.206897  3.540404
AZ       25.0   59.0         84  31.428571  3.510395
CA      329.0  779.0       1108  31.304152  3.581017
CO       25.0   78.0        103  28.873786  3.465064
CT       24.0   59.0         83  31.698795  3.584154
DC       12.0   33.0         45  30.977778  3.690329
DE        6.0    9.0         15  30.866667  3.577266
FL       42.0  120.0        162  33.456790  3.525362
GA       17.0   75.0         92  31.619565  3.525895
GU        NaN    1.0          1  25.000000  4.179351
HI        8.0   16.0         24  34.041667  3.481153
IA       13.0   44.0         57  32.614035  3.750968
ID        5.0 

Caculate genre average rating(overall and by gender)/amount of ratings/audience occupation distribution

In [187]:
pivot_by_Genre = pd.DataFrame(data.pivot_table('Rating', index = ['genres'], columns = ['Gender', 'timestamps'], aggfunc = 'mean'))
# pivot_by_Genre.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, Action to Western
Data columns (total 8 columns):
(F, 2000)    18 non-null float64
(F, 2001)    18 non-null float64
(F, 2002)    18 non-null float64
(F, 2003)    18 non-null float64
(M, 2000)    18 non-null float64
(M, 2001)    18 non-null float64
(M, 2002)    18 non-null float64
(M, 2003)    18 non-null float64
dtypes: float64(8)
memory usage: 1.3+ KB


In [27]:
pivot_by_Genre = data.pivot_table('Rating', index = ['genres'], columns = ['Gender'], aggfunc = 'count')
# pivot_by_Genre_1 = data.pivot_table('Rating', index = ['genres'], columns = ['timestamps'], aggfunc = 'mean')
#pivot_by_Genre = data.pivot_table('Rating', index = ['genres'], aggfunc = 'mean') 
# pivot_by_Genre.to_csv("pivot1.csv")
# pivot_by_Genre_1.to_csv('pivot2.csv')

In [28]:
pivot_by_Genre.head()
pivot_by_Genre.to_csv('genre_wordcloud.csv')

In [221]:
#genres = pd.read_csv("Genres.csv")

In [223]:

genres.drop(columns = ['Unnamed: 36'])
genres.head()

,Genre,F 2000,F 2001,F 2002,F 2003,M 2000,M 2001,M 2002,M 2003,Total 2000,...,12,13,14,15,16,17,18,19,20,Unnamed: 36
0,Action,3.500844,3.390796,3.373970,3.382166,3.501760,3.379891,3.277695,3.377432,3.501601,...,6.66%,1.08%,5.27%,2.52%,4.61%,9.06%,1.24%,1.44%,4.86%,NaN
1,Adventure,3.524554,3.407773,3.340330,3.454545,3.478028,3.362482,3.256757,3.374502,3.487386,...,6.76%,1.21%,4.88%,2.58%,4.50%,8.91%,1.34%,1.42%,5.20%,NaN
2,Animation,3.756576,3.647887,3.642857,3.590909,3.677121,3.517271,3.539634,3.342105,3.699404,...,6.25%,0.76%,4.02%,2.37%,3.73%,7.54%,0.86%,1.26%,6.12%,NaN
3,Children's,3.585109,3.500942,3.367491,3.406780,3.366852,3.303615,3.236818,3.156716,3.430909,...,5.41%,0.88%,3.99%,2.19%,3.76%,7.29%,1.06%,1.46%,5.92%,NaN
4,Comedy,3.588171,3.452412,3.420848,3.257225,3.512761,3.439933,3.339522,3.452769,3.532937,...,5.52%,1.22%,5.02%,2.18%,4.43%,6.89%,1.17%,1.53%,6.20%,NaN


Calculate the average rating, number of ratings for each individual movie

In [230]:
movies_info = data.pivot_table('Rating', index = ['Title'], columns = ['Gender'], aggfunc = 'mean')
movies_info['Rating'] = data.pivot_table('Rating', index = ['Title'], aggfunc = 'mean')
movies_info['No.'] = data.pivot_table('Rating', index = ['Title'], aggfunc = 'count')


In [231]:
movies_info.head()
movies_info.to_csv("movies_info.csv")

In [239]:
movies_info = pd.read_csv("movies_info.csv")
original = pd.read_csv("movies.csv")

movies_info = pd.merge(movies_info, original)

movies_info = movies_info.drop(columns = ['Unnamed: 0'])
movies_info.head()

,Title,F,M,Rating,No.,MovieID,Genres
0,"$1,000,000 Duck (1971)",3.375000,2.761905,3.027027,74,2031,Children's|Comedy
1,'Night Mother (1986),3.388889,3.352941,3.371429,70,3112,Drama
2,'Til There Was You (1997),2.675676,2.733333,2.692308,104,779,Drama|Romance
3,"'burbs, The (1989)",2.793478,2.962085,2.910891,303,2072,Comedy
4,...And Justice for All (1979),3.828571,3.689024,3.713568,398,3420,Drama|Thriller


In [240]:
movies_info.to_csv("movies_info.csv")

In [244]:
data.head()

pandas.core.frame.DataFrame

In [255]:
data_romance = data[(data['genres']=='Romance')]
data_romance.head()

,UserID,MovieID,Rating,timestamps,index,Gender,Age,OccupationID,State,Title,year,genres
8335,26,3,2,2000,25,M,25,7,VA,Grumpier Old Men (1995),1995,Romance
8337,45,3,2,2000,44,F,45,16,CA,Grumpier Old Men (1995),1995,Romance
8339,62,3,3,2000,61,F,35,3,WA,Grumpier Old Men (1995),1995,Romance
8341,137,3,5,2001,136,F,45,6,TX,Grumpier Old Men (1995),1995,Romance
8343,153,3,5,2000,152,M,1,10,IA,Grumpier Old Men (1995),1995,Romance


 Extract top 50 popular movie

In [283]:


movie_stats = data.groupby('Title').agg({'Rating': [np.size, np.mean]})
atleast_100 = movie_stats['Rating']['size'] >= 100
movie_stats = movie_stats[atleast_100].sort_values([('Rating', 'mean')], ascending=False)[:50]

In [284]:
movie_stats.to_csv("top50.csv")

In [376]:
data_str = open('USA.json').read()
data_list = json.loads(data_str)
usa = json_normalize(data_list)
usa.features[0][0]['properties']

{'adm1_code': 'USA-3513',
 'OBJECTID_1': 2059,
 'diss_me': 3513,
 'adm1_cod_1': 'USA-3513',
 'iso_3166_2': 'US-MA',
 'wikipedia': 'http://en.wikipedia.org/wiki/Massachusetts',
 'iso_a2': 'US',
 'adm0_sr': 6,
 'name': 'Massachusetts',
 'name_alt': 'Commonwealth of Massachusetts|MA|Mass.',
 'name_local': None,
 'type': 'State',
 'type_en': 'State',
 'code_local': 'US25',
 'code_hasc': 'US.MA',
 'note': None,
 'hasc_maybe': None,
 'region': 'Northeast',
 'region_cod': None,
 'provnum_ne': 0,
 'gadm_level': 1,
 'check_me': 20,
 'scalerank': 2,
 'datarank': 1,
 'abbrev': 'Mass.',
 'postal': 'MA',
 'area_sqkm': 0.0,
 'sameascity': -99,
 'labelrank': 0,
 'featurecla': 'Admin-1 scale rank',
 'name_len': 13,
 'mapcolor9': 1,
 'mapcolor13': 1,
 'fips': 'US25',
 'fips_alt': None,
 'woe_id': 2347580,
 'woe_label': 'Massachusetts, US, United States',
 'woe_name': 'Massachusetts',
 'latitude': 42.3739,
 'longitude': -71.9993,
 'sov_a3': 'US1',
 'adm0_a3': 'USA',
 'adm0_label': 2,
 'admin': 'United S

Convert the csv into json file

In [379]:
data_str = open('state_user.json').read()
data_list = json.loads(data_str)
stats= json_normalize(data_list)
stats.Genres[0][0]

{'State': 'AE',
 'F': '2.0',
 'M': '1.0',
 'TotalUser': '3',
 'Age': '38.333333333333336',
 'Rating': '3.6616008105369806'}

In [383]:
for i in stats.Genres[0]:
    for j in usa.features[0]:

        if i['State'] == j['properties']['postal']:
            
            j['properties']['F'] = i['F']
            j['properties']['M'] = i['M']
            j['properties']['User'] = i['TotalUser']
            j['properties']['Age'] = i['Age']
            j['properties']['Rating'] = i['Rating']
        

In [384]:
usa.features[0][0]['properties']

{'adm1_code': 'USA-3513',
 'OBJECTID_1': 2059,
 'diss_me': 3513,
 'adm1_cod_1': 'USA-3513',
 'iso_3166_2': 'US-MA',
 'wikipedia': 'http://en.wikipedia.org/wiki/Massachusetts',
 'iso_a2': 'US',
 'adm0_sr': 6,
 'name': 'Massachusetts',
 'name_alt': 'Commonwealth of Massachusetts|MA|Mass.',
 'name_local': None,
 'type': 'State',
 'type_en': 'State',
 'code_local': 'US25',
 'code_hasc': 'US.MA',
 'note': None,
 'hasc_maybe': None,
 'region': 'Northeast',
 'region_cod': None,
 'provnum_ne': 0,
 'gadm_level': 1,
 'check_me': 20,
 'scalerank': 2,
 'datarank': 1,
 'abbrev': 'Mass.',
 'postal': 'MA',
 'area_sqkm': 0.0,
 'sameascity': -99,
 'labelrank': 0,
 'featurecla': 'Admin-1 scale rank',
 'name_len': 13,
 'mapcolor9': 1,
 'mapcolor13': 1,
 'fips': 'US25',
 'fips_alt': None,
 'woe_id': 2347580,
 'woe_label': 'Massachusetts, US, United States',
 'woe_name': 'Massachusetts',
 'latitude': 42.3739,
 'longitude': -71.9993,
 'sov_a3': 'US1',
 'adm0_a3': 'USA',
 'adm0_label': 2,
 'admin': 'United S

In [385]:
fileObject = open('sampleList.txt', 'w')
for ip in usa.features[0]:
    fileObject.write(str(ip))
    fileObject.write(',' + '\n')
fileObject.close()